<a href="https://colab.research.google.com/github/AyrtonMastrangelo/1st-Repository/blob/main/EA3_TP5%2C_Mastrangelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EA3 - TP5 Elementos Bidimensionales

In [1]:
class LagrangeElement():    # ElementoLagrange es la clase que definira al elemento en análisis, se le deberá dar cierta información que se explica en help() en la línea 25 en adelante.
  def __init__(self, t, b, h, material, EL, ET, nu, Glt):
    self.t = t    # Espesor del elemento [m]
    self.b = b    # Altura del elemento [m]
    self.h = h    # Ancho del elemento [m]
    self.material = material
    self.EL = EL    # Módulo de Elasticidad Longitudinal [Pa]
    self.ET = ET    # Módulo de Elasticidad Transversal [Pa]
    self.nu = nu    # Coeficiente de Poisson
    self.Glt = Glt    # Módulo de Corte [Pa]


  def help(self):   # help() es una función que explica qué datos requiere VigaPortico()
    print(' LagrangeElement: se solicita la geometría del problema y datos del material.')
    print('\n t = Inserte el espesor del elemento en metros [m]')
    print('\n b = Inserte la altura del elemento en metros [m]')
    print('\n h = Inserte el ancho del elemento en metros [m]')
    print('\n material = Inserte el material a utilizar')
    print('\n EL = Inserte el módulo de elasticidad longitudinal del material en Pascales [Pa]')
    print('\n ET = Inserte el módulo de elasticidad transversal del material en Pascales [Pa]')
    print('\n nu = Inserte el valor del coeficiente de Poisson para su material')
    print('\n Glt = Inserte el módulo de corte del material en Pascales [Pa]')
    print('\n')
    print('\n LoadVector: parámetros  ')
    print('\n Nx: Carga máxima axial [N]')
    print('\n bi: valor inicial del segmento de carga distribuida en el elemento [m]')
    print('\n bf: valor final del segmento de carga distribuida en el elemento [m]')
    print('\n bl: módulo del segmento total de carga distribuida [m]')


  def data(self):   # data() te muestra los datos ingresados en VigaPortico y otros que calcula la misma clase VigaPortico internamente
    import numpy as np
    print(f' El espesor del elemento es: {self.t} m')
    print(f'\n La altura del elemento es: {self.b} m')
    print(f'\n El ancho del elemento es: {self.h} m')
    print(f'\n El material del elemento es: {self.material}')
    print(f'\n El módulo de elasticidad longitudinal del material es: {self.EL} Pa')
    print(f'\n El módulo de elasticidad transversal del material es: {self.ET} Pa')
    print(f'\n El coeficiente de Poisson del material es: {self.nu}')
    print(f'\n El módulo de corte del material es: {self.Glt} Pa')


  def K_matrix(self): # K_matrix() es la matriz de rigidez local del elemento
   
    import numpy as np
    #np.set_printoptions(precision=3, suppress=True)
    import sympy as sp
    from sympy import symbols
    x = symbols('x')
    y = symbols('y')
    x, y = symbols('x y')

    self.N1 = (1 - x/self.h)*(1 - y/self.b)
    self.N2 = (x/self.h)*(1 - y/self.b)
    self.N3 = (1 - x/self.h)*(y/self.b)
    self.N4 = (x/self.h)*(y/self.b)

    print(f' N1: {self.N1} ')
    print(f'\n N2: {self.N2} ')
    print(f'\n N3: {self.N3} ')
    print(f'\n N4: {self.N4} ')
    
    self.N = np.array([ [self.N1,   0,          self.N2,    0,          self.N3,    0,          self.N4,    0      ],
                        [0,         self.N1,    0,          self.N2,    0,          self.N3,    0,          self.N4] ])    # Matriz de polinomios de interpolación
    print('\n Matriz de polinomios de interpolación : ')
    print(f'{self.N}')
    
    B = np.array([ [sp.diff(self.N1, x),     0,                      sp.diff(self.N2, x),    0,                      sp.diff(self.N3, x),    0,                      sp.diff(self.N4, x),    0                   ],
                        [0,                       sp.diff(self.N1, y),    0,                      sp.diff(self.N2, y),    0,                      sp.diff(self.N3, y),    0,                      sp.diff(self.N4, y) ],
                        [sp.diff(self.N1, y),     sp.diff(self.N1, x),    sp.diff(self.N2, y),    sp.diff(self.N2, x),    sp.diff(self.N3, y),    sp.diff(self.N3, x),    sp.diff(self.N4, y),    sp.diff(self.N4, x)      ]   ])    # Matriz B
    print('\n Matriz B: ')
    print(f'{B}')

    D = 1/(1 - pow(self.nu, 2)) * np.array([  [self.EL,           self.nu*self.ET,    0                             ],
                                              [self.nu*self.EL,   self.ET,            0                             ],
                                              [0,                 0,                  self.Glt*(1 - pow(self.nu, 2))]  ])   # Matriz de Hooke
    print('\n Matriz de Hooke: ')
    print(f'{D}')

    #self.Klocal = sp.integrate(sp.integrate(np.dot(np.transpose(B), np.dot(D, B))*self.t, (x, 0, self.h)), (y, 0, self.b))
    BTDBt = np.dot(np.transpose(B), np.dot(D, B))*self.t
    #print(BTDBt)
    Klocal = np.zeros((len(BTDBt), len(BTDBt[0])))
    for m in range(len(BTDBt)):
      for n in range(len(BTDBt[0])):
        Klocal[(m - 1), (n - 1)] = sp.integrate(sp.integrate(BTDBt[(m - 1), (n - 1)], (x, 0, self.h)), (y, 0, self.b))
    self.Klocal = Klocal
    print('\n Matriz de rigidez local: ')
    print(f'{self.Klocal}')
    return self.Klocal


  def LoadVector(self, Nx, bi, bf, bl):    # Vector "fq" o de carga distribuidad
    '''
    Nx: Carga máxima axial
    bi: valor inicial del segmento de carga distribuida en el elemento
    bf: valor final del segmento de carga distribuida en el elemento
    bl: módulo del segmento total de carga distribuida
    '''
    self.Nx = Nx
    self.bi = bi
    self.bf = bf
    self.bl = bl
    import numpy as np
    import sympy as sp
    from sympy import symbols

    q_matrix = (-Nx/(self.t*self.bl))*np.array([[1], [0]])
    print(' Matriz q: ')
    print(q_matrix)

    x = symbols('x')
    y = symbols('y')
    x, y = symbols('x y')

    N1c = self.N1.subs(x, 0)
    N2c = self.N2.subs(x, 0)
    N3c = self.N3.subs(x, 0)
    N4c = self.N4.subs(x, 0)
    
    Nc = np.array([ [N1c,   0,      N2c,    0,      N3c,    0,      N4c,    0  ],
                    [0,     N1c,    0,      N2c,    0,      N3c,    0,      N4c]  ])    # Matriz de polinomios de interpolación
    print('\n Matriz de polinomios de interpolación : ')
    print(f'{Nc}')

    NTqt = np.dot(np.transpose(Nc), q_matrix*self.t)
    print('\n Matriz NTqt: ')
    print(f'{NTqt}')

    fq = np.zeros((len(NTqt), len(NTqt[0])))
    for m in range(len(NTqt)):
      for n in range(len(NTqt[0])):
        fq[m, n] = sp.integrate(NTqt[m, n], (y, bi, bf))
    self.fq = fq
    print('\n Matriz fq: ')
    print(f'{self.fq}')
    return self.fq


  def global_K_matrix(self, i, j, n):    # Cambia los grados de libertad del sistema del elemento al sistema global
    '''
    i: lista de grados de libertad locales.
    j: lista de grados de libertad globales.
    n: cantidad de grados de libertad del sistema.
    '''
    import numpy as np
    Kglob = np.zeros((n, n))
    for m in range(len(i)):
      for n in range(len(j)):
        Kglob[(j[m] - 1), (j[n] - 1)] = self.Klocal[(i[m] - 1), (i[n] - 1)]
    self.Kglob = Kglob
    print(f'{self.Kglob}')
    return self.Kglob


  def global_fq_matrix(self, i, j, n):    # Cambia los grados de libertad del sistema del elemento al sistema global
    '''
    i: lista de grados de libertad locales.
    j: lista de grados de libertad globales.
    n: cantidad de grados de libertad del sistema.
    '''
    import numpy as np
    fqglob = np.zeros((n, 1))
    for m in range(len(i)):
        fqglob[(j[m] - 1), 0] = self.fq[(i[m] - 1), 0]
    self.fqglob = fqglob
    print(f'{self.fqglob}')
    return self.fqglob

### Elemento 1

In [2]:
elemento_1 = LagrangeElement(0.00175, 0.0865, 0.54, 'Carbono/Epoxy', 70000000000, 70000000000, 0.1, 5000000000)

In [3]:
K1 = elemento_1.K_matrix()

 N1: (1 - 1.85185185185185*x)*(1 - 11.5606936416185*y) 

 N2: 1.85185185185185*x*(1 - 11.5606936416185*y) 

 N3: 11.5606936416185*y*(1 - 1.85185185185185*x) 

 N4: 21.4086919289231*x*y 

 Matriz de polinomios de interpolación : 
[[(1 - 1.85185185185185*x)*(1 - 11.5606936416185*y) 0
  1.85185185185185*x*(1 - 11.5606936416185*y) 0
  11.5606936416185*y*(1 - 1.85185185185185*x) 0 21.4086919289231*x*y 0]
 [0 (1 - 1.85185185185185*x)*(1 - 11.5606936416185*y) 0
  1.85185185185185*x*(1 - 11.5606936416185*y) 0
  11.5606936416185*y*(1 - 1.85185185185185*x) 0 21.4086919289231*x*y]]

 Matriz B: 
[[21.4086919289231*y - 1.85185185185185 0
  1.85185185185185 - 21.4086919289231*y 0 -21.4086919289231*y 0
  21.4086919289231*y 0]
 [0 21.4086919289231*x - 11.5606936416185 0 -21.4086919289231*x 0
  11.5606936416185 - 21.4086919289231*x 0 21.4086919289231*x]
 [21.4086919289231*x - 11.5606936416185
  21.4086919289231*y - 1.85185185185185 -21.4086919289231*x
  1.85185185185185 - 21.4086919289231*y
  11.560693

In [4]:
fq1 = elemento_1.LoadVector(1250, 0.075, 0.0865, 0.023)

 Matriz q: 
[[-31055900.62111801]
 [       -0.        ]]

 Matriz de polinomios de interpolación : 
[[1 - 11.5606936416185*y 0 0 0 11.5606936416185*y 0 0 0]
 [0 1 - 11.5606936416185*y 0 0 0 11.5606936416185*y 0 0]]

 Matriz NTqt: 
[[628298.567479266*y - 54347.8260869565]
 [0]
 [0]
 [0]
 [-628298.567479266*y]
 [0]
 [0]
 [0]]

 Matriz fq: 
[[ -41.54624277]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [-583.45375723]
 [   0.        ]
 [   0.        ]
 [   0.        ]]


In [5]:
K1glob = elemento_1.global_K_matrix([1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], 12)

[[ 2.48150572e+07  5.28093434e+06  2.49708153e+06  9.05934343e+05
  -1.49046101e+07 -9.05934343e+05 -1.24075286e+07 -5.28093434e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.28093434e+06  2.57955383e+08 -9.05934343e+05  1.28276881e+08
   9.05934343e+05 -2.57254573e+08 -5.28093434e+06 -1.28977692e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.49708153e+06 -9.05934343e+05  2.48150572e+07 -5.28093434e+06
  -1.24075286e+07  5.28093434e+06 -1.49046101e+07  9.05934343e+05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.05934343e+05  1.28276881e+08 -5.28093434e+06  2.57955383e+08
   5.28093434e+06 -1.28977692e+08 -9.05934343e+05 -2.57254573e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.49046101e+07  9.05934343e+05 -1.24075286e+07  5.28093434e+06
   2.48150572e+07 -5.28093434e+06  2.49708153e+06 -9.05934343e+05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.

In [6]:
fq1glob = elemento_1.global_fq_matrix([1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], 12)

[[ -41.54624277]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [-583.45375723]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]]


### Elemento 2

In [7]:
elemento_2 = LagrangeElement(0.00175, 0.0865, 0.54, 'Carbono/Epoxy', 70000000000, 70000000000, 0.1, 5000000000)

In [8]:
K2 = elemento_2.K_matrix()

 N1: (1 - 1.85185185185185*x)*(1 - 11.5606936416185*y) 

 N2: 1.85185185185185*x*(1 - 11.5606936416185*y) 

 N3: 11.5606936416185*y*(1 - 1.85185185185185*x) 

 N4: 21.4086919289231*x*y 

 Matriz de polinomios de interpolación : 
[[(1 - 1.85185185185185*x)*(1 - 11.5606936416185*y) 0
  1.85185185185185*x*(1 - 11.5606936416185*y) 0
  11.5606936416185*y*(1 - 1.85185185185185*x) 0 21.4086919289231*x*y 0]
 [0 (1 - 1.85185185185185*x)*(1 - 11.5606936416185*y) 0
  1.85185185185185*x*(1 - 11.5606936416185*y) 0
  11.5606936416185*y*(1 - 1.85185185185185*x) 0 21.4086919289231*x*y]]

 Matriz B: 
[[21.4086919289231*y - 1.85185185185185 0
  1.85185185185185 - 21.4086919289231*y 0 -21.4086919289231*y 0
  21.4086919289231*y 0]
 [0 21.4086919289231*x - 11.5606936416185 0 -21.4086919289231*x 0
  11.5606936416185 - 21.4086919289231*x 0 21.4086919289231*x]
 [21.4086919289231*x - 11.5606936416185
  21.4086919289231*y - 1.85185185185185 -21.4086919289231*x
  1.85185185185185 - 21.4086919289231*y
  11.560693

In [9]:
fq2 = elemento_2.LoadVector(1250, 0, 0.0115, 0.023)

 Matriz q: 
[[-31055900.62111801]
 [       -0.        ]]

 Matriz de polinomios de interpolación : 
[[1 - 11.5606936416185*y 0 0 0 11.5606936416185*y 0 0 0]
 [0 1 - 11.5606936416185*y 0 0 0 11.5606936416185*y 0 0]]

 Matriz NTqt: 
[[628298.567479266*y - 54347.8260869565]
 [0]
 [0]
 [0]
 [-628298.567479266*y]
 [0]
 [0]
 [0]]

 Matriz fq: 
[[-583.45375723]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [ -41.54624277]
 [   0.        ]
 [   0.        ]
 [   0.        ]]


In [10]:
K2glob = elemento_2.global_K_matrix([1, 2, 3, 4, 5, 6, 7, 8], [5, 6, 7, 8, 9, 10, 11, 12], 12)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.48150572e+07  5.28093434e+06  2.49708153e+06  9.05934343e+05
  -1.49046101e+07 -9.05934343e+05 -1.24075286e+07 -5.28093434e+06]
 [ 0.

In [11]:
fq2glob = elemento_2.global_fq_matrix([1, 2, 3, 4, 5, 6, 7, 8], [5, 6, 7, 8, 9, 10, 11, 12], 12)

[[   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [-583.45375723]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [ -41.54624277]
 [   0.        ]
 [   0.        ]
 [   0.        ]]


### Matriz de Rigidez Global $K_{glob}$

In [12]:
Kglob = K1glob + K2glob    # Se suman las matrices globales de cada elemento para obtener la matriz de rigidez global del sistema
print(Kglob)

[[ 2.48150572e+07  5.28093434e+06  2.49708153e+06  9.05934343e+05
  -1.49046101e+07 -9.05934343e+05 -1.24075286e+07 -5.28093434e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.28093434e+06  2.57955383e+08 -9.05934343e+05  1.28276881e+08
   9.05934343e+05 -2.57254573e+08 -5.28093434e+06 -1.28977692e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.49708153e+06 -9.05934343e+05  2.48150572e+07 -5.28093434e+06
  -1.24075286e+07  5.28093434e+06 -1.49046101e+07  9.05934343e+05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.05934343e+05  1.28276881e+08 -5.28093434e+06  2.57955383e+08
   5.28093434e+06 -1.28977692e+08 -9.05934343e+05 -2.57254573e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.49046101e+07  9.05934343e+05 -1.24075286e+07  5.28093434e+06
   4.96301144e+07  9.31322575e-10  4.99416307e+06  2.32830644e-10
  -1.49046101e+07 -9.05934343e+05 -1.24075286e+07 -5.28093434e+06]
 [-9.

### Matriz de Cargas Nodales $f_{q}$

In [13]:
fqglob = fq1glob + fq2glob    # Se suman las matrices globales de cada elemento para obtener la matriz de cargas nodales global del sistema
print(fqglob)

[[  -41.54624277]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [-1166.90751445]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [  -41.54624277]
 [    0.        ]
 [    0.        ]
 [    0.        ]]


### Reducción de Guyán: *Condensación Estática*

El método reordena los grados de libertad, de forma que los desplazamientos desconocidos se ubiquen en los primeros grandos de libertad. Mientras que con las cargas ocurre lo opuesto.

In [14]:
def guyan_K_matrix(k, l, h):    # Función que cambia los grados de libertad del sistema global al sistema transformado por Guyán
  '''
  k: lista de grados de libertad globales.
  l: lista de grados de libertad de Guyán.
  h: cantidad de grados de libertad del sistema.
  '''
  import numpy as np
  Kguyan = np.zeros((h, h))
  for g in range(len(k)):
    for h in range(len(l)):
      Kguyan[(l[g] - 1), (l[h] - 1)] = Kglob[(k[g] - 1), (k[h] - 1)]
  Kguyan = Kguyan
  print(f'{Kguyan}')
  return Kguyan

In [15]:
Kguyan = guyan_K_matrix([1,2,5,6,3,4,7,8,9,10,11,12], [1,2,3,4,5,6,7,8,9,10,11,12],  12)

[[ 2.48150572e+07  5.28093434e+06 -1.49046101e+07 -9.05934343e+05
   2.49708153e+06  9.05934343e+05 -1.24075286e+07 -5.28093434e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.28093434e+06  2.57955383e+08  9.05934343e+05 -2.57254573e+08
  -9.05934343e+05  1.28276881e+08 -5.28093434e+06 -1.28977692e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.49046101e+07  9.05934343e+05  4.96301144e+07  9.31322575e-10
  -1.24075286e+07  5.28093434e+06  4.99416307e+06  2.32830644e-10
  -1.49046101e+07 -9.05934343e+05 -1.24075286e+07 -5.28093434e+06]
 [-9.05934343e+05 -2.57254573e+08 -1.86264515e-09  5.15910767e+08
   5.28093434e+06 -1.28977692e+08 -2.32830644e-10  2.56553763e+08
   9.05934343e+05 -2.57254573e+08 -5.28093434e+06 -1.28977692e+08]
 [ 2.49708153e+06 -9.05934343e+05 -1.24075286e+07  5.28093434e+06
   2.48150572e+07 -5.28093434e+06 -1.49046101e+07  9.05934343e+05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.

In [16]:
def guyan_fq_matrix(k, l, h):    # Función que cambia los grados de libertad del sistema global al sistema transformado por Guyán
  '''
  k: lista de grados de libertad globales.
  l: lista de grados de libertad de Guyán.
  h: cantidad de grados de libertad del sistema.
  '''
  import numpy as np
  fqguyan = np.zeros((h, 1))
  for g in range(len(k)):
      fqguyan[(l[g] - 1), 0] = fqglob[(k[g] - 1), 0]
  fqguyan = fqguyan
  print(f'{fqguyan}')
  return fqguyan

In [17]:
fqguyan = guyan_fq_matrix([1,2,5,6,3,4,7,8,9,10,11,12], [1,2,3,4,5,6,7,8,9,10,11,12],  12)

[[  -41.54624277]
 [    0.        ]
 [-1166.90751445]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [  -41.54624277]
 [    0.        ]
 [    0.        ]
 [    0.        ]]


### Sistema de Ecuaciones

\begin{equation}
    \begin{Bmatrix}
  P
  \end{Bmatrix}
  =
  \begin{bmatrix}
  K
  \end{bmatrix}
  ∙
  \begin{Bmatrix}
  U
  \end{Bmatrix}
  +
  \begin{Bmatrix}
  f_{q}
  \end{Bmatrix}
  \Rightarrow
  \begin{Bmatrix}
  P_{I}\\
  P_{II}
  \end{Bmatrix}
  =
  \begin{bmatrix}
  K_{I-I} & K_{I-II}\\
  K_{II-I} & K_{II-II}
  \end{bmatrix}
  ∙
  \begin{Bmatrix}
  U_{I}\\
  U_{II}
  \end{Bmatrix}
    -
  \begin{Bmatrix}
  f_{q_{I}}\\
  f_{q_{II}}
  \end{Bmatrix}
\end{equation}


Las cargas y desplazamientos a calcular son $P_{II}$ y $U_{I}$ respectivamente.


In [18]:
KI_I = Kguyan[0:4, 0:4]
KI_II = Kguyan[0:4, 4:12]
KII_I = Kguyan[4:12, 0:4]
KII_II = Kguyan[4:12, 4:12]
fqI = fqguyan[0:4, 0]
fqII = fqguyan[4:12, 0]

#np.set_printoptions(precision=3, suppress=True)
print(f'Partición de la matriz de rigidez global para resolver el sistema matricial de ecuaciones: ')

print(f'\n KI_I ')
rows = len(KI_I)
columns = len(KI_I[0])
print(f'\n Cantidad de filas: {rows} ')
print(f'\n Cantidad de columnas: {columns} ')
print(f'\n {KI_I} ')

print(f'\n KI_II ')
rows = len(KI_II)
columns = len(KI_II[0])
print(f'\n Cantidad de filas: {rows} ')
print(f'\n Cantidad de columnas: {columns} ')
print(f'\n {KI_II} ')

print(f'\n KII_I ')
rows = len(KII_I)
columns = len(KII_I[0])
print(f'\n Cantidad de filas: {rows} ')
print(f'\n Cantidad de columnas: {columns} ')
print(f'\n {KII_I} ')

print(f'\n KII_II ')
rows = len(KII_II)
columns = len(KII_II[0])
print(f'\n Cantidad de filas: {rows} ')
print(f'\n Cantidad de columnas: {columns} ')
print(f'\n {KII_II} ')

print(f'\n Partición de la matriz de cargas nodales para resolver el sistema matricial de ecuaciones: ')

print(f'\n fqI ')
rows = len(fqI)
#columns = len(fqI[0])
print(f'\n Cantidad de filas: {rows} ')
print(f'\n Cantidad de columnas: {columns} ')
print(f'\n {fqI} ')

print(f'\n fqII ')
rows = len(fqII)
#columns = len(fqII[0])
print(f'\n Cantidad de filas: {rows} ')
print(f'\n Cantidad de columnas: {columns} ')
print(f'\n {fqII} ')

Partición de la matriz de rigidez global para resolver el sistema matricial de ecuaciones: 

 KI_I 

 Cantidad de filas: 4 

 Cantidad de columnas: 4 

 [[ 2.48150572e+07  5.28093434e+06 -1.49046101e+07 -9.05934343e+05]
 [ 5.28093434e+06  2.57955383e+08  9.05934343e+05 -2.57254573e+08]
 [-1.49046101e+07  9.05934343e+05  4.96301144e+07  9.31322575e-10]
 [-9.05934343e+05 -2.57254573e+08 -1.86264515e-09  5.15910767e+08]] 

 KI_II 

 Cantidad de filas: 4 

 Cantidad de columnas: 8 

 [[ 2.49708153e+06  9.05934343e+05 -1.24075286e+07 -5.28093434e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.05934343e+05  1.28276881e+08 -5.28093434e+06 -1.28977692e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.24075286e+07  5.28093434e+06  4.99416307e+06  2.32830644e-10
  -1.49046101e+07 -9.05934343e+05 -1.24075286e+07 -5.28093434e+06]
 [ 5.28093434e+06 -1.28977692e+08 -2.32830644e-10  2.56553763e+08
   9.05934343e+05 -2.57254573e+08 -5.28093434e+06 

$U_{I}=k_{I-I}^{-1}∙(P_{I}-k_{I-II}∙U_{II}+f_{q_{I}})$

$P_{II}=k_{II-I}∙U_{I}+k_{II-II}∙U_{II}-f_{q_{II}}$

In [19]:
#np.set_printoptions(precision=15, suppress=True)
#np.set_printoptions(formatter={'float': lambda x:'%0.12f'%(x)})
import numpy as np
from numpy.linalg import inv
KI_Iinv = inv(KI_I)
print(f' Matriz K_I-I inversa: ')
print(f'\n {KI_Iinv}')

 Matriz K_I-I inversa: 

 [[ 4.96605543e-08 -1.95379870e-09  1.49494157e-08 -8.87041864e-10]
 [-1.95379870e-09  7.78928696e-09 -7.28936269e-10  3.88063171e-09]
 [ 1.49494157e-08 -7.28936269e-10  2.46518791e-08 -3.37226921e-10]
 [-8.87041864e-10  3.88063171e-09 -3.37226921e-10  3.87180648e-09]]


In [20]:
#np.set_printoptions(precision=20, suppress=True)
UII = np.transpose(np.array([0, 0, 0, 0, 0, 0, 0, 0]))    # Matriz UII
print(' Vector de desplazamientos conocidos UII: ')
print(f'\n {UII} m')

 Vector de desplazamientos conocidos UII: 

 [0 0 0 0 0 0 0 0] m


In [21]:
#np.set_printoptions(precision=1, suppress=True)
PI = np.transpose(np.array([0, 0, 0, 0])) # Matriz PI
print(' Vector de cargas conocidas PI: ')
print(f'\n {PI} N')

 Vector de cargas conocidas PI: 

 [0 0 0 0] N


In [22]:
#np.set_printoptions(precision=10, suppress=True)
UI = np.dot(KI_Iinv, PI - np.dot(KI_II, UII) + fqI)   # Matriz UI
print(' Los desplazamientos desconocidos son: ')
print(f'\n {UI} m')

 Los desplazamientos desconocidos son: 

 [-1.95077950e-05  9.31774205e-07 -2.93875551e-05  4.30365885e-07] m


In [23]:
#np.set_printoptions(formatter={'float': lambda x:'%0.3f'%(x)})
PII = np.dot(KII_I, UI) + np.dot(KII_II, UII) - fqII   # Matriz PII
print(' Las cargas desconocidas son: ')
print(f'\n {PII} N')

 Las cargas desconocidas son: 

 [ 317.34298301 -108.84903954   90.3566439    93.2532859   479.94617709
  -84.09039674  362.35419599   99.68615038] N
